In [2]:
import fnv
import fnv.reduce
import fnv.file  
import numpy as np
import seaborn as sns
import datetime as dt
import time
import pandas as pd
from tkinter import filedialog
from tkinter import *
import os
import matplotlib.pyplot as plt
import glob
import h5py
import cv2

h5py.run_tests()

...........s...........................................x...........................s.....s......................................x....................................sC:\Users\cloftus\Anaconda3\lib\site-packages\h5py\_hl\files.py:258: H5pyDeprecationWarning: File.fid has been deprecated. Use File.id instead.
  "Use File.id instead.", H5pyDeprecationWarning)
...s......ss.ss...............................................................................ss...................ssssss.....................................................................x....x.........................x......x..................................................ssss...................s........................................
----------------------------------------------------------------------
Ran 509 tests in 3.566s

OK (skipped=22, expected failures=6)


<unittest.runner.TextTestResult run=509 errors=0 failures=0>

In [5]:
# root = Tk()
# root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
# print (root.filename)

In [6]:
datelist = [ d.date() for d in pd.date_range(start = pd.to_datetime('2019-08-06'), end = pd.to_datetime('2019-08-07')).to_pydatetime()]
print(datelist)

[datetime.date(2019, 8, 6), datetime.date(2019, 8, 7)]


In [ ]:

## hard coded writing from raw files to a given video with a start and end timestamp
# timestamps are not rounded. Every frame is written as is between start_timestamp and end_timestmp

zoom = 5

drive = 'E:/'

start_timestamp = '2019_08_08_15_44_29_67'
end_timestamp = '2019_08_08_16_23_08_514'


write_to = 'C:/Users/cloftus/Desktop/vids_for_prediction/' + start_timestamp


start_dt = dt.datetime.strptime(start_timestamp, '%Y_%m_%d_%H_%M_%S_%f')
end_dt = dt.datetime.strptime(end_timestamp, '%Y_%m_%d_%H_%M_%S_%f')

curr_date = f'{start_dt.year:04}' + f'{start_dt.month:02}' + f'{start_dt.day:02}'

os.chdir( drive + curr_date )

file_name = glob.glob('*.seq')

im = fnv.file.ImagerFile(file_name[0])

im.get_frame(0)

day1 = int(im.frame_info[0]['value'][:3])

os.makedirs(write_to)
os.chdir(write_to)


for i in range(im.num_frames):
    
    im.get_frame(i)

    day2 = int(im.frame_info[0]['value'][:3])

    day_diff = day2 - day1

    timestamp = f'{start_dt.year:04}-' + f'{start_dt.month:02}-' + f'{(start_dt.day+day_diff):02} ' + im.frame_info[0]['value'][4:]

    timestamp = dt.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
    
    rounded_stamp = round_to_fifth( timestamp )
    
    if (rounded_stamp >= start_dt) & (rounded_stamp <= end_dt):

        final = np.array(im.final, copy=False).reshape((im.height, im.width))

        final1 = final - final.mean()

        fig, ax = plt.subplots(figsize = (final.shape[1] / (100/zoom), final.shape[0] / (100/zoom)))  

        ax = sns.heatmap(final1, vmin = -np.std(final1), vmax = final1.max(), xticklabels = False, yticklabels = False, cbar = False)

        if final1.shape[1] > 1000:

            plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 55*zoom / 5 )

        else:

            plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 25*zoom / 5 )

        plt.savefig( timestamp.strftime("%Y%m%d_%H%M%S%f") + '.tiff' , bbox_inches = 'tight')

        plt.close()
        
        


In [7]:
### helpful bit of code that renames files so they have a counter in front of the timestamp

root_path = 'C:/Users/cloftus/Desktop/Results/'

os.chdir(root_path)

folders = os.listdir()

for folder in folders:
        
        os.chdir(root_path + folder)
    
        files = os.listdir()
    
        counter = 0
        
        for file in files:
            
            if 'tiff' in file:
                #print( file.split( '_', 1) )
                #print( file.split( '_', 1)[1] )
                #os.rename( file,   file.split( '_', 1)[1] )
                #os.rename( file,   file[4: ] )
                #os.rename( file,  f'{counter:05}' + '_' + file )
                
                counter += 1

In [13]:
## makes a txt file for each folder in Results/ of all of the tiff images in that folder 
## this txt file will be used by ffmpeg in the command line to string the pictures together into a video

root_path = 'C:/Users/cloftus/Desktop/Results/'

os.chdir(root_path)

folders = os.listdir()

for folder in folders:
        
    os.chdir(root_path + folder)
    
    files = os.listdir()
    
    text_file = open(folder + '.txt', 'w')
        
    for file in files:

        if 'tiff' in file:

            n = text_file.write('file ' + "'" + file + "'" + '\n')

    text_file.close()